In [1]:
# verified on 2/16/2025
# format of pass.txt
"""
google:your_email@gmail.com:your_google_app_password
yahoo:your_email@yahoo.com:your_yahoo_app_password
""";

In [2]:
import imaplib
import email
from email.header import decode_header
import pandas as pd
import re

# Yahoo IMAP settings
IMAP_SERVER = 'imap.mail.yahoo.com'
IMAP_PORT = 993
# EMAIL = 'your_email@yahoo.com'
# PASSWORD = 'your_app_password'

with open('pass.txt') as fin:
    strs=fin.readlines()
for s in strs:
    if s.split(":")[0]=='yahoo':
        EMAIL=s.split(":")[1].strip()
        PASSWORD=s.split(":")[-1].strip()
# EMAIL,PASSWORD    

# get emails

In [3]:
# Connect to the server
mail = imaplib.IMAP4_SSL(IMAP_SERVER, IMAP_PORT)
mail.login(EMAIL, PASSWORD)

MAILBOX='INBOX'

status, _ = mail.select(MAILBOX)
if status != 'OK':
    print(f"Failed to select mailbox: {MAILBOX}")

In [4]:

start_date = '19-Feb-2025'
end_date = '18-Feb-2025'

search_criteria=f'(SINCE {start_date})'

status, messages = mail.uid('search',None, search_criteria)

# Search for emails within the date range
# status, messages = mail.uid('search',None, f'(SINCE {start_date} BEFORE {end_date})')
# status, messages = mail.search(None, f'SINCE "{start_date}"')
# status, messages = mail.search(None, 'ALL')

# sender_email = 'example@domain.com'
# # Search for emails from a particular address between dates
# search_criteria = f'(FROM "{sender_email}" SINCE "{start_date}" BEFORE "{end_date}")'
# status, messages = mail.search(None, search_criteria)

# Convert the message IDs to a list
target_uids = messages[0].split()

print(f"Found {len(target_uids)} emails.")

Found 38 emails.


In [5]:
results=[]
i=0
for uid in target_uids:
    i+=1
    # UID FETCH command
    status, data = mail.uid('fetch', uid, '(RFC822)')
    if status == 'OK' and data != [None]:
        r={}
        # Parse email content
        raw_email = data[0][1]
        msg = email.message_from_bytes(raw_email)
        sender=msg['From']
        print(f"\n{i} UID: {uid}")
        print(f"From: {sender}")
        
        # Decode headers
        subject, encoding = decode_header(msg['Subject'])[0]
        if isinstance(subject, bytes):
            try:
                subject = subject.decode(encoding if encoding else 'utf-8')
            except:
                subject="Decoding Error"        

        print(f"Subject: {subject}")

        # Extract the date
        email_date = msg["Date"]
        print(email_date)
        
        r["id"]=uid
        r["sender"]=sender
        r["date"]=email_date
        r["subject"]=subject
        results.append(r)           
    else:
        print(f"Email with UID {uid} not found")


1 UID: b'582794'
From: Alameda County <acgov@service.govdelivery.com>
Subject: Alameda County Property Tax News & Announcements - 2/18/2025
Wed, 19 Feb 2025 01:23:01 +0000

2 UID: b'582798'
From: =?UTF-8?B?QWxhc2thIEFpcmxpbmVz?= <deals@ifly.alaskaair.com>
Subject: Sale alert! Fares start at $49 or 4,000 miles one way
Wed, 19 Feb 2025 02:04:06 +0000

3 UID: b'582801'
From: <tax.payment@acgov.org>
Subject: Self Service Tax Transaction Acknowledgement
Tue, 18 Feb 2025 18:28:19 -0800

4 UID: b'582814'
From: "Bank of America" <onlinebanking@ealerts.bankofamerica.com>
Subject: Your Available Balance
Wed, 19 Feb 2025 05:37:43 -0600

5 UID: b'582821'
From: "Charles Schwab & Co., Inc." <donotreply@email.schwab.com>
Subject: Social Security and taxes, Medicare premiums, and more
Wed, 19 Feb 2025 07:13:30 -0600

6 UID: b'582846'
From: USPS Informed Delivery <USPSInformeddelivery@email.informeddelivery.usps.com>
Subject: Your Daily Digest for Wed, 2/19 is ready to view
Wed, 19 Feb 2025 15:27:33 +

In [6]:

# Close connection
mail.close()
mail.logout()

('BYE', [b'IMAP4rev1 Server logging out'])

# process email

In [7]:
def get_email(str_sender):
    pattern = re.compile(r'<(.*?)>')
    match = re.search(pattern, str_sender)

    if match:
        # print("Found:", match.group())
        email=match.group(1)
    else:
        # print("Not found")
        email="NA"   

    sender, encoding = decode_header(str_sender)[0]
    if isinstance(sender, bytes):
        sender = sender.decode(encoding if encoding else 'utf-8')

    return email,sender   

In [8]:
df=pd.DataFrame(results)
df[["email","sender"]]=df.apply(lambda r:get_email(r['sender']),axis=1,result_type='expand')

In [9]:
df

,id,sender,date,subject,email
0,b'582794',Alameda County <acgov@service.govdelivery.com>,"Wed, 19 Feb 2025 01:23:01 +0000",Alameda County Property Tax News & Announcemen...,acgov@service.govdelivery.com
1,b'582798',Alaska Airlines,"Wed, 19 Feb 2025 02:04:06 +0000","Sale alert! Fares start at $49 or 4,000 miles ...",deals@ifly.alaskaair.com
2,b'582801',<tax.payment@acgov.org>,"Tue, 18 Feb 2025 18:28:19 -0800",Self Service Tax Transaction Acknowledgement,tax.payment@acgov.org
3,b'582814',"""Bank of America"" <onlinebanking@ealerts.banko...","Wed, 19 Feb 2025 05:37:43 -0600",Your Available Balance,onlinebanking@ealerts.bankofamerica.com
4,b'582821',"""Charles Schwab & Co., Inc."" <donotreply@email...","Wed, 19 Feb 2025 07:13:30 -0600","Social Security and taxes, Medicare premiums, ...",donotreply@email.schwab.com
5,b'582846',USPS Informed Delivery <USPSInformeddelivery@e...,"Wed, 19 Feb 2025 15:27:33 +0000","Your Daily Digest for Wed, 2/19 is ready to view",USPSInformeddelivery@email.informeddelivery.us...
6,b'582870',"""Chevrolet Certified Service"" <Chevrolet@deliv...","Wed, 19 Feb 2025 11:43:26 -0500",Time for an oil change? Save up to $30,Chevrolet@delivery.Chevrolet.com
7,b'582903',noreply@lhh.com,"Wed, 19 Feb 2025 19:01:08 +0000 (UTC)",Optimize Your Job Search: Reminder Email,NA
8,b'582930',"""Facebook"" <groupupdates@facebookmail.com>","Wed, 19 Feb 2025 12:38:44 -0800",Zhi Feng added a photo in 美國房屋裝修交流分享,groupupdates@facebookmail.com
9,b'582934',Fandango <fandango@movies.fandango.com>,"Wed, 19 Feb 2025 21:01:57 +0000 (UTC)",Your $5 Is Waiting For You,fandango@movies.fandango.com


In [10]:
xx=list(df.email.unique())
xx

['acgov@service.govdelivery.com',
 'deals@ifly.alaskaair.com',
 'tax.payment@acgov.org',
 'onlinebanking@ealerts.bankofamerica.com',
 'donotreply@email.schwab.com',
 'USPSInformeddelivery@email.informeddelivery.usps.com',
 'Chevrolet@delivery.Chevrolet.com',
 'NA',
 'groupupdates@facebookmail.com',
 'fandango@movies.fandango.com',
 'friends@facebookmail.com',
 'no-reply@accounts.google.com',
 'info@account.netflix.com',
 'merlyns@mail.packtpub.com',
 'no-reply@e.siriusxm.com',
 'marriottvacationclub@email1.marriott-vacations.com',
 'venmo@email.venmo.com',
 'store+28212789326@m.shopifyemail.com',
 'engineering@our.berkeley.edu',
 'dancer@bicluart.com',
 'strategy@insights.FinancesDataDriven.com',
 'just@info.globalfinancechanges.com',
 'newsletter@a.bigmarketwatchers.com',
 'learn@global.1marketface.com',
 'news@ldii.longdistanceinvestings.com',
 'news@tee.theeconomicecho.com',
 'invest@with.theretirementresearch.com',
 'news@wrrn.wealthyrepublicnews.com']

In [11]:
df.groupby(['email']).count().reset_index().sort_values('subject').tail(10)

,email,id,sender,date,subject
25,strategy@insights.FinancesDataDriven.com,1,1,1,1
19,news@wrrn.wealthyrepublicnews.com,1,1,1,1
20,newsletter@a.bigmarketwatchers.com,1,1,1,1
22,no-reply@e.siriusxm.com,1,1,1,1
24,store+28212789326@m.shopifyemail.com,1,1,1,1
1,NA,2,2,2,2
21,no-reply@accounts.google.com,2,2,2,2
9,friends@facebookmail.com,3,3,3,3
10,groupupdates@facebookmail.com,4,4,4,4
23,onlinebanking@ealerts.bankofamerica.com,4,4,4,4


In [12]:
f2=['temu@updates.temuofficial.com', 'mylife.com',
   'americanstockadvisors',
   'thestockstandout','valoraeclipse','tradingfusionlab','metaversecapitalists','financialstoryarc',
   'bayareanewsgroup','recordstrivia','graynavigator',
   'investmentinnovate.com',
   'theinvestingtime.com',
   'a.richmarketwaves.com',
   'quantumcapitalvision',
   'invest','market','fortune','needtoknowfacts','fiscalforecastnet','milkymoneyway','quantumcapitalvision','fiscalforecastnet','financialstoryarc'
   'thestockstandout'
   'readysteadyprofit.com',
   'taxfoundation.com',
   'theretirementresearch.com',
   'customink.com',
   'oldamericanbroker.com',
   'smartsuccessstrategy.com',
   'theinvestingbox.com',
   'texasinnovationsmarket.com',
   'investingwithmind.com',
   'directionforexpo.com',
   'teamexpoexpert.com',
   'fiscalforecastnet',
   'r.moore@first.AmericanTruthCast.com', 
   'insight@the.paybackinvestigators.com',
   'edmond.p@daily.smoothtradingarea.com',
   'move@like.dealscatchers.com', 
   'news@tradelikegorillas.com',
   'ruffus.white@net.richpeoplenetworks.com',
   'raymondparks@tmp.titanmediapro.com', 
   'ae@e.ae.com',
   'daily@news.greenforesttrading.com',
   'daily@the.bankerbull.com', 
   'daily@blog.secretprofittools.com',
   'day@s.longliveamerican.com', 
#    'read@first.teamexpoexpert.com',
   'peter.s@the.directionforexpo.com', 
   'daily@inside.greenforesttrading.com',
   'l.barnett@wtf.wealthtimefuture.com',
   'email@digital.conservativeintel.com', 'gain@the.olympinvests.com',
   'hunter.wells@tip.profitabletradertips.com',
   'check@tips.sportinvestorsleague.com',
   'actual@Ii.investmentinnovate.com',
   'editor@nowcc.notesonwealthcreation.com',
   'editorial@wsi.wisetradeinvesting.com',
   'news@fn.fin-kit.com',
   'info@tfm.thefinancememories.com', 
   'news@daily.financiffy.com',
   'mailing@ceg.certaineconomicgrowth.com',
   'today@art.americansruletrading.com',
   'daily@mcp.moderncryptoparadise.com',
   'daily@d.tradetacticsnavigator.com',
#    'j.blume@mc.metaversecapitalists.com',
   'news@from.thegreatassets.com', 
   'ronaldvin@a.capitaliststoday.com']

In [13]:
f1=['walmartcustomerexperience@express10.medallia.com',
    #    'fandango@movies.fandango.com',
       'ftowdklvjeqagcszbymr@plugins.homemakingguide.com',
       'gain@the.olympinvests.com', 
    #    'noreplyhr@fb.com',
    #    'Costco@digital.costco.com', 
       'raymondparks@tmp.titanmediapro.com',
       'read@first.teamexpoexpert.com',
       'e-news@email.bayareanewsgroup.com',
    #    'billpay@billpay.bankofamerica.com', 
    #    'no-reply@rentcafe.com',
    #    'onlinebanking@ealerts.bankofamerica.com',
    #    'store+28212789326@m.shopifyemail.com',
    #    'hello@news.expressvpn.com', 
    #    'NA', 
    #    'palmia@legacypartners.com',
       'editorial@wsi.wisetradeinvesting.com',
    #    'groupupdates@facebookmail.com', 
       'me@jasongee.site',
       'silhruettrgrzphsextup@plugins.homemakingguide.com',
       'contact@masterclassjournal.com',
       'notifications@d.lavendercollab.com', 
       'humble@pobriar.com',
       'kent.harper@news.theinvestingbox.com',
       'cutleryandmore@cutleryandmore.com', 
       'contact@lectricebikes.com',
       'trivia@mail.wordstrivia.com',
       'just@info.globalfinancechanges.com',
       'global@news.takeprofitalerts.com',
       'email@the.deliciousinvesting.com', 
       'reply-to@e.digikey.com',
    #    'hongxiachen@hotmail.com', 
       'p.lynch@hmg.honeymoneygazette.com',
       'daily@the.bankerbull.com', 
       'news@s.stripesandstarsnews.com',
       'mylife@mail.mylife.com', 
       'marketing@emails.hertz.com',
       'move@like.dealscatchers.com',
    #    'j.blume@mc.metaversecapitalists.com',
       'from@global.investingcongress.com',
       'editor@star.starinvestingship.com',
       'daily@your.financeinsidemedia.com',
       'news@tee.theeconomicecho.com',
       'new@readysteadyprofit.com',
       'editor@tddy.tradersdynasty.com',
       'news@t.keyincomejournal.com',
       'news@wrrn.wealthyrepublicnews.com',
       'gain@wfp.wealthfuturepath.com',
       'growth@get.cashfortify.com',
       'info@cferfoundation.org',
       'nextdish@nextdish.com',
       'eyelid@prisintol.com',
       'insights@from.tradingrealities.com',
       'news@us.nationmonetization.com',
       'elizabethharris@community.uptradingiq.com',
       'thh@welcome.tradehavenhub.com',
       'get@real.multitradersmoves.com',
       'your@sas.successamericanstories.com',
       'strategy@insights.FinancesDataDriven.com',
       'christopher.johnson@ucc.urbancapitalist.com',
       'daily@day.usastocksinsider.com',
       'growth@your.impressiverevenue.com',
       'weatherproof@weircpot.com',
       'mail@your.usagoldmiracle.com',
       'robertfalco@fin.finacetechfuture.com',
       'info@promo8.appewa.com',
       'pr@fischerconnectors.info',
       'media@top.yourprioritydeal.com',
       'fremont-police-department-ca@emails.nixle.com',
       'empowered@sellvos.com',
       'deals@deals.vegas.com',
       'boldface@vortpact.com',
       'sunflower@nikerse.com',
       'anybody@tolivren.com',
       'mailfrom@cottonbubblelay.com',
       'euphoria@qermotal.com',
       'noreply@komoot.de',
       'swardeacnessarphisr@plugins.homemakingguide.com',
       'phthplderiliennizaqw@plugins.homemakingguide.com',
       'support@g.sapphireguru.com',       
       'announcements@mkt.databricks.com',
       't.killian@sotmm.sharksofthemarket.com']

In [14]:
xlist=[]
for x in xx:
    fInlcuded=False
    for y in f1+f2:
        if y in x:
            fInlcuded=True
    if not fInlcuded:
        xlist.append(x) 
xlist

['acgov@service.govdelivery.com',
 'deals@ifly.alaskaair.com',
 'tax.payment@acgov.org',
 'onlinebanking@ealerts.bankofamerica.com',
 'donotreply@email.schwab.com',
 'USPSInformeddelivery@email.informeddelivery.usps.com',
 'Chevrolet@delivery.Chevrolet.com',
 'NA',
 'groupupdates@facebookmail.com',
 'fandango@movies.fandango.com',
 'friends@facebookmail.com',
 'no-reply@accounts.google.com',
 'info@account.netflix.com',
 'merlyns@mail.packtpub.com',
 'no-reply@e.siriusxm.com',
 'marriottvacationclub@email1.marriott-vacations.com',
 'venmo@email.venmo.com',
 'store+28212789326@m.shopifyemail.com',
 'engineering@our.berkeley.edu',
 'dancer@bicluart.com']

In [15]:
mustKeepList=['NA',
 'yibingtang@gmail.com',
 'ybtang02@yahoo.com',
 'hongxiachen@hotmail.com',
 'groupupdates@facebookmail.com',
 'reminders@facebookmail.com',
 'store-news@amazon.com',
 'PayPal@emails.paypal.com',
 'no-reply@accounts.google.com',
 'sail_internet@birdeye.com',
 'onlinebanking@ealerts.bankofamerica.com',
 'no-reply@e.siriusxm.com',
 'no-reply@em-s.dropbox.com',
 'USPSInformeddelivery@email.informeddelivery.usps.com',
 'no-reply@email.alltrails.com',
 'Costco@digital.costco.com',
 'mail@eg.expedia.com',
 'Online-Do-Not-Reply@dmv.ca.gov',
 'cmueng-advancement@connect.cmu.edu',
 'echosign@echosign.com',
 'news@scientificamerican.com',
 'SchwabStockPlanServices@schwab.com',
 'marriottbonvoy@email-marriott.com',
 'acgov@service.govdelivery.com',
 'deals@ifly.alaskaair.com',
 'tax.payment@acgov.org',
 'donotreply@email.schwab.com',
 'Chevrolet@delivery.Chevrolet.com',
 'fandango@movies.fandango.com',
 'friends@facebookmail.com',
 'info@account.netflix.com',
 'merlyns@mail.packtpub.com',
 'marriottvacationclub@email1.marriott-vacations.com',
 'venmo@email.venmo.com',
 'store+28212789326@m.shopifyemail.com',
 'engineering@our.berkeley.edu']

In [16]:

email_to_delete=f1+f2+['blog.secretprofittools.com','pro.milkymoneyway.com','ldii.longdistanceinvestings.com','first.AmericanTruthCast.com','today.investingwithmind.com','day.usastocksinsider.com',"best.marketexpopro.com",'net.richpeoplenetworks.com','fin.finacetechfuture.com','editor@nowcc.notesonwealthcreation.com','prisintol.com','tfm.thefinancememories.com','today.investingwithmind.com','ucc.urbancapitalist.com','digest.marketbroadview.com','tips.sportinvestorsleague.com','cottonbubblelay.com','deals@deals.vegas.com','find@via.healthyinvestingtips.com','webinar@newsletters.techonline.com','l.barnett@wtf.wealthtimefuture.com','get@real.multitradersmoves.com','mailing@ceg.certaineconomicgrowth.com','temu@updates.temuofficial.com','strategy@insights.FinancesDataDriven.com','ruffus.white@net.richpeoplenetworks.com','with@top.goldmineadvisors.com','your@very.smartsuccessstrategy.com','news@daily.investmentjourneys.com','check@tips.sportinvestorsleague.com','daily@digest.marketbroadview.com','q.hayes@choice.besttoinvestnews.com','news@daily.financiffy.com','daily@blog.secretprofittools.com','mail@your.usagoldmiracle.com','the.directionforexpo.com','daily@digest.marketbroadview.com']


In [17]:
print(len(email_to_delete))
email_to_delete=[x for x in email_to_delete if x not in mustKeepList]
print(len(email_to_delete))

172
172


In [18]:
subject_to_delete =['David Kant','FunWords Team','richard grant','stockpile saving','satelite earnings now','this token could','David Kant','Nextdish','hotels.com', 'Matthew Wehner','Elizabeth Harris', 'Trump','David Kant','Part-Time Work','Investors League','MileagePlus Partner','Russia in Shock','woot.com','Christopher Johnson','Tax-Free Cash Hack']

In [19]:
pattern1 = '|'.join(subject_to_delete)
pattern2 = '|'.join(email_to_delete)

In [20]:

dz=df[(df.sender.str.contains(pattern1, case=False)) | (df.email.str.contains(pattern2, case=False))]

In [21]:
dy=df[(~df.sender.str.contains(pattern1, case=False)) & (~df.email.str.contains(pattern2, case=False))]
dy.groupby(['email']).count().reset_index().sort_values('subject').tail(10)

,email,id,sender,date,subject
13,merlyns@mail.packtpub.com,1,1,1,1
12,marriottvacationclub@email1.marriott-vacations...,1,1,1,1
18,tax.payment@acgov.org,1,1,1,1
19,venmo@email.venmo.com,1,1,1,1
17,store+28212789326@m.shopifyemail.com,1,1,1,1
1,NA,2,2,2,2
14,no-reply@accounts.google.com,2,2,2,2
9,friends@facebookmail.com,3,3,3,3
10,groupupdates@facebookmail.com,4,4,4,4
16,onlinebanking@ealerts.bankofamerica.com,4,4,4,4


In [ ]:
import time
testdate=time.strftime("%Y%m%d-%H%M")
dz[['email','subject','sender','id']].sort_values(by='email').to_csv(f'yahoo_email_delete_record_{testdate}.csv')

In [ ]:
dz.shape

(8, 5)

In [23]:
uids_to_delete=list(dz.id.unique())
uids_to_delete, len(uids_to_delete)

([b'583084',
  b'583085',
  b'583086',
  b'583087',
  b'583088',
  b'583089',
  b'583090',
  b'583091'],
 8)

# delete emails

In [24]:
# Connect to the server
mail = imaplib.IMAP4_SSL(IMAP_SERVER, IMAP_PORT)
mail.login(EMAIL, PASSWORD)

MAILBOX='INBOX'

status, _ = mail.select(MAILBOX)
if status != 'OK':
    print(f"Failed to select mailbox: {MAILBOX}")

In [25]:
 
# delete emails
i=0
for uid in uids_to_delete:
    i+=1
    print(i,uid)
    try:
        status, response = mail.uid('store', uid, '+FLAGS', '\\Deleted')
        print(f"Delete status: {status}, Response: {response}")
    except:
        print(uid,"not working")
        
# Expunge deleted emails
mail.expunge()

1 b'583084'
Delete status: OK, Response: [b'9993 (FLAGS (\\Deleted \\Seen $NotJunk) UID 583084)']
2 b'583085'
Delete status: OK, Response: [b'9993 (FLAGS (\\Deleted \\Seen $NotJunk) UID 583085)']
3 b'583086'
Delete status: OK, Response: [b'9993 (FLAGS (\\Deleted \\Seen $NotJunk) UID 583086)']
4 b'583087'
Delete status: OK, Response: [b'9993 (FLAGS (\\Deleted \\Seen $NotJunk) UID 583087)']
5 b'583088'
Delete status: OK, Response: [b'9993 (FLAGS (\\Deleted \\Seen $NotJunk) UID 583088)']
6 b'583089'
Delete status: OK, Response: [b'9993 (FLAGS (\\Deleted \\Seen $NotJunk) UID 583089)']
7 b'583090'
Delete status: OK, Response: [b'9993 (FLAGS (\\Deleted \\Seen $NotJunk) UID 583090)']
8 b'583091'
Delete status: OK, Response: [b'9993 (FLAGS (\\Deleted \\Seen $NotJunk) UID 583091)']


('OK',
 [b'9993', b'9993', b'9993', b'9993', b'9993', b'9993', b'9993', b'9993'])

In [26]:

# Close connection
mail.close()
mail.logout()

('BYE', [b'IMAP4rev1 Server logging out'])